In [1]:
import matplotlib.pyplot as plt
import time 
import numpy as np
from astropy.io import fits
import os 

In [2]:
directorio='Ingrese su directorio' #Creacion del directorio donde se ubican las imagenes
contenido = os.listdir(directorio)

# PROCESO DE DEMULTIPLEXADO

In [121]:
# Loop sobre todas las imagenes, se guarda en img_CDD16 cada imagen en nsamp/div archivos correspondientes a 16 CCD c/u.
st = time.time()
img_CCD16=fits.HDUList([])

for item in range(0,len(contenido)): #Recorrido sobre las imagenes en RAW (imagenes crudas)
    hdulist = fits.open(directorio+contenido[item])
    tamx=int(hdulist[4].header['NAXIS1']) 
    tamy=int(hdulist[4].header['NAXIS2']) 
    nsamp=int(hdulist[4].header['NSAMP']) 
    ncol=int(hdulist[4].header['NCOL'])
    ccdncol=int(hdulist[4].header['CCDNCOL'])
    scidata = hdulist[4].data
    tamxpimg=int(tamx/nsamp)
    div=16 #Variable multiplo de 2^n 
    
    #Proceso que recorre las imagenes en el directorio, se separa una imagen en archivos de lectura de 16 CCD c/u.
    #Dichas imagenes van sobreescribiendose dentro de una lista para su posterior guardado como nuevo archivo.
    for j in range(0,int(nsamp/16)):                         
        img_parcial= np.zeros(tamy,tamxpimg,dtype='i4')   
        for i in range(0,div):                                 
            for k in range(0,tamxpimg):              
                img_parcial[:,k+i*tamxpimg]=scidata[:,(k*nsamp)+(i+div*j)] 
    #Proceso en el que se elimina el offset de cada imagen en base al analisis de la zona de overscan.
    #Darle una solucion/optimizacion al proceso de guardado
            for p in range(tamy):
                offset=np.mean(img_parcial[p,(tamxpimg-int(ncol-ccdncol/2)):])
                img_parcial[p,:]=img_individual[p,:]-offset
        img_CCD16.append(fits.ImageHDU(img_parcial))        
        
    #Proceso de guardado en el que se crean las nuevas imagenes.
    nombre=str("Imagen"+str(item)+".fits")
    hdul=fits.HDUList(img_CCD16)    
    hdul.writeto(nombre,overwrite=True)
    print('Se termino la imagen ',item)
    img_CCD16.clear()
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
#Resultado parcial de momento: Se obtienen 7 archivos por cada imagen de OSCURA los cuales corresponden a 16 CCD cada uno,
#los mismos archivos ya no poseen offset y deben de calcularse aun su ganancia, procesar dichos valores y cargarlos
#a una posterior base de datos. 

Se termino la imagen  0
Se termino la imagen  1
Se termino la imagen  2
Se termino la imagen  3
Se termino la imagen  4
Execution time: 17.37893772125244 seconds


# PROCESO DE PARALELIZACION

In [ ]:
import concurrent.futures

In [ ]:
#Prueba de paralelizado
#Funcion a paralelizar demux
def demux(nombre):
    print("Comienza a demultiplexar \n")
    hdulist = fits.open(nombre)
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    img_total= np.zeros((tamy, tamx))
    tamxpimg=int(tamx/nsamp)
    
    for i in range(nsamp):
        for k in range (tamxpimg):
            img_total[:,k+tamxpimg*i]=scidata[:,(k*nsamp)+i]
    print('Se termino la imagen ') 
    

In [ ]:
type(hdulist)

In [ ]:
##Main
if __name__ == '__main__':    
    directorio='Ingrese su directorio' 
    contenido = os.listdir(directorio)
    for j in range(0,len(contenido)):
        contenido[j]=directorio+contenido[j]
        
    t1 = time.perf_counter()
    #with concurrent.futures.ProcessPoolExecutor() as executor:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(demux, contenido)
    t2 = time.perf_counter()
    print(f'Finished in {t2-t1} seconds')


In [ ]:
from numba import njit,prange

In [ ]:
#Funcion a optimizar
#Necesita trabajar solo con arrays
@njit(parallel=True)
def demux2(tamx,tamy,nsamp,scidata):
    img_total= np.zeros((tamy, tamx))
    tamxpimg=int(tamx/nsamp)
    for i in prange(nsamp):##se puede paralelizar este
        
        for k in range (tamxpimg):
            img_total[:,k+tamxpimg*i]=scidata[:,(k*nsamp)+i]
            
    print('Se termino la imagen ')   


In [ ]:
directorio='Ingrese su directorio' 
contenido = os.listdir(directorio) 
for j in range(0,len(contenido)):
    contenido[j]=directorio+contenido[j]
    
st = time.time()
for iterador in range(0,len(contenido)): #For para cada item del directorio
    hdulist = fits.open(contenido[iterador])
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    demux2(tamx,tamy,nsamp,scidata)
et = time.time()
elapsed_time = et - st
print('First Execution time :', elapsed_time, 'seconds')

st = time.time()
for iterador in range(0,len(contenido)): #For para cada item del directorio
    hdulist = fits.open(contenido[iterador])
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    demux2(tamx,tamy,nsamp,scidata)

et = time.time()
elapsed_time = et - st
print('Second Execution time :', elapsed_time, 'seconds')